In [14]:
from mIAm.graph.workflow import workflow
from mIAm.graph.trimmer import TRIMMER
import os 
import psycopg
from psycopg_pool import AsyncConnectionPool
from langgraph.checkpoint.postgres.aio import AsyncPostgresSaver
from langchain_core.messages import HumanMessage, AIMessage

In [9]:
DB_URI = f"postgresql://{os.getenv('PSQL_USERNAME')}:{os.getenv('PSQL_PASSWORD')}" \
         f"@{os.getenv('PSQL_HOST')}:{os.getenv('PSQL_PORT')}/{os.getenv('PSQL_DATABASE')}" \
         f"?sslmode={os.getenv('PSQL_SSLMODE')}"
            
connection_kwargs = {
    "autocommit": True,
    "prepare_threshold": 0,
}

user_config = {
    "first_name": "Branis",
    "last_name": "GHOUL",
    "email": "branisghoul02@hotmail.com",
    "password": "123456789",
    "phone": "123456789",
    "address": "30 rue victor hugo",
    "city": "Paris",
    "country": "France",
    "zip_code": "75001",
    "language": ["fr", "en", "ar"],
    "timezone": "Europe/Paris",
    "currency": "EUR",
}



In [ ]:
async with AsyncConnectionPool(
    conninfo=DB_URI,
    max_size=20,
    kwargs=connection_kwargs
) as pool:
    checkpointer = AsyncPostgresSaver(pool)

    # NOTE: you need to call .setup() the first time you're using your checkpointer
    await checkpointer.setup()
    graph = workflow.compile(checkpointer=checkpointer)
    config = {"configurable": {"thread_id": "1"}}
    while True:
        user = input("User (q/Q to quit): ")
        print(f"User (q/Q to quit): {user}")
        if user in {"q", "Q"}:
            print("AI: Byebye")
            break
        output = None
        async for output in graph.astream(
            {"messages": [HumanMessage(content=user)]}, config=config, stream_mode="updates"
        ):
            last_message = next(iter(output.values()))["messages"][-1]
            last_message.pretty_print()

        if output and "prompt" in output:
            print("Done!")
            

           
    checkpoint = await checkpointer.aget(config)

User (q/Q to quit): hello how are you
================================== Ai Message ==================================

Hello! I'm here and ready to help you with any culinary questions or recipe needs you might have. How can I assist you today? 🍽️👩‍🍳
User (q/Q to quit): q
AI: Byebye


In [23]:
async with AsyncConnectionPool(
    conninfo=DB_URI,
    max_size=20,
    kwargs=connection_kwargs
) as pool:
    checkpointer = AsyncPostgresSaver(pool)

    # NOTE: you need to call .setup() the first time you're using your checkpointer
    await checkpointer.setup()
    config = {"configurable": {"thread_id": "1"}}
    checkpoint = await checkpointer.aget(config)
trimmed_message = TRIMMER.invoke(checkpoint["channel_values"]["messages"])
len(trimmed_message), trimmed_message

checkpoint["channel_values"]["messages"]
# filter only AIMessage
filtered = [x.content for x in checkpoint["channel_values"]["messages"] if isinstance(x, AIMessage)]
filtered

['Hello! 🍽️ How can I assist you in the kitchen today? Are you looking for a recipe, cooking tips, or something else related to food? Let me know!',
 "Hey there! 👋 What's cooking? Are you looking for a recipe or some culinary advice? Let me know how I can help! 🍳🥗",
 'It seems like your message might not have come through. If you have any questions or need assistance with cooking, feel free to let me know! 🍽️✨',
 'Couscous with Loubia Kabyle sounds like a delicious and hearty dish! To help you with this recipe, could you please provide a bit more information?\n\n- **Dietary Preferences:** Do you have any dietary restrictions or preferences?\n- **Available Ingredients:** What ingredients do you have on hand?\n- **Time Constraints & Skill Level:** How much time do you have to prepare this dish, and what is your cooking experience level?\n- **Special Requests:** Any specific flavors or techniques you want to include?\n\nWith this information, I can provide a tailored recipe for you! 🍲🌿',
